<a href="https://colab.research.google.com/github/hildazaqya/project-capstone/blob/main/ML/Foodection%20%20Model%20Experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#this file from kaggle API personnal account
from google.colab import files

# Upload kaggle.json
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"smjayyy","key":"a86a230a085a77cff7a5851725589584"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
##link dataset in Kaggle.com
!kaggle datasets download -d alinesellwia/food-freshness

 99% 2.66G/2.67G [00:27<00:00, 169MB/s]
100% 2.67G/2.67G [00:27<00:00, 103MB/s]


In [4]:
#new directory for stores temporary data
!mkdir data

In [5]:
#unzip the data
!unzip -q food-freshness.zip -d /content/data

Resize Gambar Untuk Satu Pixel Yang Sama

In [6]:
from PIL import Image
import os

def resize_images_in_directory(directory, target_size_size):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(('.jpg', '.jpeg', '.png')):  # Sesuaikan dengan ekstensi gambar yang Anda miliki
                file_path = os.path.join(root, file)
                img = Image.open(file_path)

                if img.mode == 'RGBA':
                    img = img.convert('RGB')

                img_resized = img.resize(target_size)
                img_resized.save(file_path)

# Tentukan ukuran baru yang diinginkan, misalnya (224, 224)
target_size = (224, 224)

# Proses direktori train
resize_images_in_directory("/content/data/Foodection Dataset/train/jeruk_busuk", target_size)
resize_images_in_directory("/content/data/Foodection Dataset/train/jeruk_segar", target_size)
resize_images_in_directory("/content/data/Foodection Dataset/train/jeruk_segar_sedang", target_size)
resize_images_in_directory("/content/data/Foodection Dataset/train/tomat_busuk", target_size)
resize_images_in_directory("/content/data/Foodection Dataset/train/tomat_segar", target_size)
resize_images_in_directory("/content/data/Foodection Dataset/train/tomat_segar_sedang", target_size)
resize_images_in_directory("/content/data/Foodection Dataset/train/wortel_busuk", target_size)
resize_images_in_directory("/content/data/Foodection Dataset/train/wortel_segar", target_size)
resize_images_in_directory("/content/data/Foodection Dataset/train/wortel_segar_sedang", target_size)

# Proses direktori test
resize_images_in_directory("/content/data/Foodection Dataset/test/jeruk_busuk", target_size)
resize_images_in_directory("/content/data/Foodection Dataset/test/jeruk_segar", target_size)
resize_images_in_directory("/content/data/Foodection Dataset/test/jeruk_segar_sedang", target_size)
resize_images_in_directory("/content/data/Foodection Dataset/test/tomat _segar_sedang", target_size)
resize_images_in_directory("/content/data/Foodection Dataset/test/tomat_busuk", target_size)
resize_images_in_directory("/content/data/Foodection Dataset/test/tomat_segar", target_size)
resize_images_in_directory("/content/data/Foodection Dataset/test/wortel_busuk", target_size)
resize_images_in_directory("/content/data/Foodection Dataset/test/wortel_segar", target_size)
resize_images_in_directory("/content/data/Foodection Dataset/test/wortel_segar_sedang", target_size)

In [25]:
#Train Data, Augmentation
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, Model, regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D, Dropout

#Transfer Learning
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.applications import EfficientNetB0

#optimizer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop

#interach with OS -> make directory
import os

#evaluation
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

#Visualization
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

#for save model
from tensorflow.keras.models import load_model
from google.colab import files


In [8]:
base_dir= '/content/data/Foodection Dataset'

print("Contents of base directory:")
print(os.listdir(base_dir))

print("\nContents of train directory:")
print(os.listdir(f'{base_dir}/train'))

print("\nContents of validation directory:")
print(os.listdir(f'{base_dir}/test'))

Contents of base directory:
['test', 'train']

Contents of train directory:
['wortel_busuk', 'tomat_segar', 'tomat_busuk', 'jeruk_segar', 'tomat_segar_sedang', 'wortel_segar_sedang', 'wortel_segar', 'jeruk_busuk', 'jeruk_segar_sedang']

Contents of validation directory:
['wortel_busuk', 'tomat_segar', 'tomat_busuk', 'jeruk_segar', 'wortel_segar_sedang', 'wortel_segar', 'jeruk_busuk', 'jeruk_segar_sedang', 'tomat _segar_sedang']


In [9]:
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

# Specify class of the directory train
train_jeruksegar_dir = os.path.join(train_dir, 'jeruk_segar')
train_wortelsegar_dir = os.path.join(train_dir, 'wortel_segar')
train_wortelsegarsedang_dir = os.path.join(train_dir, 'wortel_segar')
train_tomatbusuk_dir = os.path.join(train_dir, 'tomat_busuk')
train_jeruksegarsedang_dir = os.path.join(train_dir, 'jeruk_segar_sedang')
train_jerukbusuk_dir = os.path.join(train_dir, 'jeruk_busuk')
train_wortelbusuk_dir = os.path.join(train_dir, 'wortel_busuk')
train_tomatsegar_dir = os.path.join(train_dir, 'tomat_segar')
train_tomatsegarsedang_dir = os.path.join(train_dir, 'tomat_segar_sedang')

# Specify class of the directory test
test_jeruksegar_dir = os.path.join(test_dir, 'jeruk_segar')
test_wortelsegar_dir = os.path.join(test_dir, 'wortel_segar')
test_wortelsegarsedang_dir = os.path.join(test_dir, 'wortel_segar_sedang')
test_tomatbusuk_dir = os.path.join(test_dir, 'tomat_busuk')
test_jeruksegarsedang_dir = os.path.join(test_dir, 'jeruk_segar_sedang')
test_jerukbusuk_dir = os.path.join(test_dir, 'jeruk_busuk')
test_wortelbusuk_dir = os.path.join(test_dir, 'wortel_busuk')
test_tomatsegar_dir = os.path.join(test_dir, 'tomat_segar')
test_tomatsegarsedang_dir = os.path.join(test_dir, 'tomat _segar_sedang')

In [10]:
train_jeruksegar_fnames = os.listdir(train_jeruksegar_dir)
train_wortelsegar_fnames = os.listdir(train_wortelsegar_dir)
train_wortelsegarsedang_fnames = os.listdir(train_wortelsegarsedang_dir)
train_tomatbusuk_fnames = os.listdir(train_tomatbusuk_dir)
train_jeruksegarsedang_fnames = os.listdir(train_jeruksegarsedang_dir)
train_jerukbusuk_fnames = os.listdir(train_jerukbusuk_dir)
train_wortelbusuk_fnames = os.listdir(train_wortelbusuk_dir)
train_tomatsegar_fnames = os.listdir(train_tomatsegar_dir)
train_tomatsegarsedang_fnames = os.listdir(train_tomatsegarsedang_dir)

In [11]:
print('total training jeruk segar images :', len(os.listdir(train_jeruksegar_dir)))
print('total training wortel segar images :', len(os.listdir( train_wortelsegar_dir)))
print('total training wortel segar sedang images :', len(os.listdir(train_wortelsegarsedang_dir)))
print('total training tomat busuk images :', len(os.listdir(train_tomatbusuk_dir)))
print('total training jeruk segar sedang images :', len(os.listdir(train_jeruksegarsedang_dir)))
print('total training jeruk busuk images :', len(os.listdir(train_jerukbusuk_dir)))
print('total training wortel busuk images :', len(os.listdir(train_wortelbusuk_dir)))
print('total training tomat segar images :', len(os.listdir(train_tomatsegar_dir)))
print('total training tomat segar sedang images :', len(os.listdir(train_tomatsegarsedang_dir)))

print('total testing jeruk segar images :', len(os.listdir(test_jeruksegar_dir)))
print('total testing wortel segar images :', len(os.listdir(test_wortelsegar_dir)))
print('total testing wortel segar sedang images :', len(os.listdir(test_wortelsegarsedang_dir)))
print('total testing tomat busuk images :', len(os.listdir(test_tomatbusuk_dir)))
print('total testing jeruk segar sedang images :', len(os.listdir(test_jeruksegarsedang_dir)))
print('total testing jeruk busuk images :', len(os.listdir(test_jerukbusuk_dir)))
print('total testing wortel busuk images :', len(os.listdir(test_wortelbusuk_dir)))
print('total testing tomat segar images :', len(os.listdir(test_tomatsegar_dir)))
print('total testing tomat segar sedang images :', len(os.listdir(test_tomatsegarsedang_dir)))

total training jeruk segar images : 3460
total training wortel segar images : 4241
total training wortel segar sedang images : 4241
total training tomat busuk images : 1115
total training jeruk segar sedang images : 2401
total training jeruk busuk images : 1658
total training wortel busuk images : 1360
total training tomat segar images : 4745
total training tomat segar sedang images : 3109
total testing jeruk segar images : 1482
total testing wortel segar images : 1818
total testing wortel segar sedang images : 966
total testing tomat busuk images : 478
total testing jeruk segar sedang images : 1029
total testing jeruk busuk images : 703
total testing wortel busuk images : 583
total testing tomat segar images : 2033
total testing tomat segar sedang images : 1332


In [12]:
img_height = 224
img_width = 224
img_channels = 3
num_epochs = 20

train_datagen = ImageDataGenerator( rotation_range=20,
  width_shift_range=0.1,
  height_shift_range=0.1,
  shear_range=0.1,
  zoom_range=0.1,
  horizontal_flip=True,
  vertical_flip=True,
  rescale=1./255,
  validation_split=0.2)

test_datagen  = ImageDataGenerator( rescale = 1.0/255.)

train_generator= train_datagen.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    color_mode='rgb',
                                                    class_mode='categorical',
                                                    target_size=(img_height, img_width))
test_generator =  test_datagen.flow_from_directory(test_dir,
                                                    batch_size=20,
                                                    color_mode='rgb',
                                                    class_mode='categorical',
                                                    target_size=(img_height, img_width))

Found 24341 images belonging to 9 classes.
Found 10422 images belonging to 9 classes.


1. Transfer Learning MobileNetV2

In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, img_channels))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Choose a specific layer for feature extraction
x = layers.Flatten()(base_model.output)
x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(9, activation='softmax')(x)  # Adjust the number of output classes

model = Model(base_model.input, x)

#early stop
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

9406464/9406464 [==============================] - 2s 0us/step


In [ ]:
history = model.fit(train_generator, epochs=num_epochs, validation_data=test_generator, callbacks=[early_stopping])

Epoch 1/20
 963/1218 [======================>.......] - ETA: 1:15 - loss: 2.7072 - accuracy: 0.6399

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


1218/1218 [==============================] - 397s 315ms/step - loss: 2.4713 - accuracy: 0.6593 - val_loss: 2.1238 - val_accuracy: 0.4928
Epoch 2/20
1218/1218 [==============================] - 380s 312ms/step - loss: 1.3844 - accuracy: 0.7366 - val_loss: 1.8944 - val_accuracy: 0.5143
Epoch 3/20
1218/1218 [==============================] - 378s 310ms/step - loss: 1.2802 - accuracy: 0.7441 - val_loss: 1.9045 - val_accuracy: 0.5585
Epoch 4/20
1218/1218 [==============================] - 377s 310ms/step - loss: 1.2400 - accuracy: 0.7503 - val_loss: 1.5517 - val_accuracy: 0.5828
Epoch 5/20
1218/1218 [==============================] - 375s 308ms/step - loss: 1.2322 - accuracy: 0.7490 - val_loss: 2.0189 - val_accuracy: 0.5552
Epoch 6/20
1218/1218 [==============================] - 380s 312ms/step - loss: 1.2541 - accuracy: 0.7353 - val_loss: 1.6789 - val_accuracy: 0.5735


2. CNN 3

In [13]:
model1 = tf.keras.models.Sequential()

model1.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model1.add(layers.MaxPooling2D((2, 2)))

model1.add(layers.Conv2D(64, (3, 3), activation='relu'))
model1.add(layers.MaxPooling2D((2, 2)))

model1.add(layers.Conv2D(128, (3, 3), activation='relu'))
model1.add(layers.MaxPooling2D((2, 2)))

model1.add(layers.Flatten())

model1.add(layers.Dense(256, activation='relu'))
model1.add(layers.Dense(9,activation='softmax'))

model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

In [14]:
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Compile the model
model1.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
history = model1.fit(train_generator, epochs=num_epochs, validation_data=test_generator, callbacks=[early_stopping])

Epoch 1/20
 163/1218 [===>..........................] - ETA: 4:43 - loss: 1.6775 - accuracy: 0.4015

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


1218/1218 [==============================] - 364s 288ms/step - loss: 1.0720 - accuracy: 0.6164 - val_loss: 2.0599 - val_accuracy: 0.4491
Epoch 2/20
1218/1218 [==============================] - 351s 288ms/step - loss: 0.6639 - accuracy: 0.7671 - val_loss: 2.1169 - val_accuracy: 0.4819
Epoch 3/20
1218/1218 [==============================] - 349s 286ms/step - loss: 0.5271 - accuracy: 0.8188 - val_loss: 2.1556 - val_accuracy: 0.5155


3. CNN 4 and Adjust Layer

In [33]:
from tensorflow.keras.applications import ResNet50V2

baseModel = ResNet50V2(weights="imagenet", include_top=False,input_shape=(img_height, img_width, img_channels))

headModel = baseModel.output
headModel = GlobalAveragePooling2D()(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(9, activation="softmax")(headModel)

model2 = Model(inputs=baseModel.input, outputs = headModel)

early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Freeze the baseModel layers
for layer in baseModel.layers :
    layer.trainable = False

model2.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=["accuracy"])
model2.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_7[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 pool1_pad (ZeroPadding2D)   (None, 114, 114, 64)         0         ['conv1_conv[0][0]']          
                                                                                            

In [34]:
history = model2.fit(train_generator, epochs=num_epochs, validation_data = test_generator, verbose=1, callbacks=[early_stopping])

Epoch 1/20
 557/1218 [============>.................] - ETA: 3:11 - loss: 0.7486 - accuracy: 0.7426

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


1218/1218 [==============================] - 391s 318ms/step - loss: 0.5898 - accuracy: 0.7966 - val_loss: 1.4428 - val_accuracy: 0.6425
Epoch 2/20
1218/1218 [==============================] - 386s 317ms/step - loss: 0.3752 - accuracy: 0.8674 - val_loss: 1.7837 - val_accuracy: 0.6302
Epoch 3/20
1218/1218 [==============================] - 388s 318ms/step - loss: 0.3345 - accuracy: 0.8795 - val_loss: 1.8091 - val_accuracy: 0.6284


In [35]:
from tensorflow.keras.applications import ResNet50V2

baseModel = ResNet50V2(weights="imagenet", include_top=False,input_shape=(img_height, img_width, img_channels))

headModel = baseModel.output
headModel = GlobalAveragePooling2D()(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(9, activation="softmax")(headModel)

model2 = Model(inputs=baseModel.input, outputs = headModel)

early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Freeze the baseModel layers
for layer in baseModel.layers :
    layer.trainable = True

model3.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=["accuracy"])
model3.summary()

Exception ignored in: <function _xla_gc_callback at 0x7ee05fcd70a0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 101, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_8[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 pool1_pad (ZeroPadding2D)   (None, 114, 114, 64)         0         ['conv1_conv[0][0]']          
                                                                                            

KeyboardInterrupt: ignored

In [ ]:
history = model3.fit(train_generator, epochs=num_epochs, validation_data = test_generator, verbose=1, callbacks=[early_stopping])